In [2]:
!pip install transformers 
!pip install sentencepiece
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 36.5 MB/s 
     |████████████████████████████████| 7.6 MB 60.3 MB/s 
     |████████████████████████████████| 182 kB 67.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 35.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import transformers
import numpy as np
import pandas as pd
import torch
import warnings
warnings.filterwarnings('ignore')
import sentencepiece

In [5]:
# For DistilBERT:
from transformers import DistilBertModel, DistilBertTokenizer, XLNetModel, XLNetTokenizer
#model_class, tokenizer_class, pretrained_weights = (DistilBertModel, DistilBertTokenizer, 'lvwerra/distilbert-imdb')

# Want BERT instead of distilBERT? Uncomment the following line:
model_class, tokenizer_class, pretrained_weights = (XLNetModel, XLNetTokenizer, 'xlnet-base-cased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/PAPI_NASONOV/lab2/movie_info_final.csv')
df = df.drop(columns=['index'])
df.head()

,imdb_id,title,plot,genres
0,4179452,The Last Kingdom,"In the year 872, many of the separate kingdoms...","['Action', 'Drama', 'History']"
1,3748528,Rogue One,All looks lost for the Rebellion against the E...,"['Action', 'Adventure', 'Sci-Fi']"
2,5626028,Boku no hîrô akademia,"In a world populated with superhumans, the sup...","['Animation', 'Action', 'Adventure', 'Fantasy'..."
3,3322312,Daredevil,As a child Matt Murdock was blinded by a chemi...,"['Action', 'Crime', 'Drama', 'Fantasy', 'Sci-F..."
4,2580046,Miraculous: Tales of Ladybug & Cat Noir,"By daylight, Marinette Dupain-Cheng and Adrien...","['Animation', 'Action', 'Adventure', 'Family',..."


In [ ]:
len(df)

3503

In [ ]:
df.iloc[0]['plot']

'In the year 872, many of the separate kingdoms of what is now England have fallen to the invading Danes, leaving the great kingdom of Wessex standing alone and defiant under the command of King Alfred. Against this turbulent backdrop lives Uhtred. Born to a Saxon nobleman, he is captured by the Danes and raised as one of their own. Forced to choose between his birth country and the people of his upbringing, his loyalties are ever tested. What is he? Saxon or Dane? On a quest to reclaim his birthright, Uhtred must tread a dangerous path between both sides to play his part in the birth of a new nation and--ultimately--recapture his ancestral lands.'

In [ ]:
len(df[3500:3600])

3

In [ ]:
batches = [df[x * 100 : (x + 1) * 100] for x in range(0,36)]

In [ ]:
model.cuda()

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [ ]:
#gets a batch of texts and returns hidden states from bert encoder on CLS slot output
def get_batch_hidden_states(batch):
  tokenized = batch.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

  max_len = 0
  for i in tokenized.values:
      if len(i) > max_len:
          max_len = len(i)

  padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

  attention_mask = np.where(padded != 0, 1, 0)

  input_ids = torch.tensor(padded).cuda()  
  attention_mask = torch.tensor(attention_mask).cuda()

  with torch.no_grad():
      last_hidden_states = model(input_ids, attention_mask=attention_mask)
  
  return last_hidden_states[0].cpu()[:,0,:].numpy()

In [ ]:
#gets a batch of texts and returns hidden states from bert encoder on CLS slot output
def get_batch_hidden_states_plus(batch):
  inputs = tokenizer.batch_encode_plus(batch,
                                       add_special_tokens=True,
                                       return_attention_mask=True,
                                       padding='longest',
                                       max_length=512,
                                       truncation=True,
                                       return_tensors='pt' )

  input_ids = inputs['input_ids'].cuda()  
  attention_mask = inputs['attention_mask'].cuda()

  with torch.no_grad():
      last_hidden_states = model(input_ids, attention_mask=attention_mask)
  
  return last_hidden_states[0].cpu()[:,0,:].numpy()

In [ ]:
from tqdm import tqdm

batches_hidden_states = []

for batch in tqdm(batches):
  batch_hidden_states = get_batch_hidden_states_plus(batch['plot'])
  batches_hidden_states.append(batch_hidden_states)


100%|██████████| 36/36 [00:56<00:00,  1.58s/it]


In [ ]:
type(batches_hidden_states[0])

numpy.ndarray

In [ ]:
total_hidden_states = np.concatenate(batches_hidden_states)
len(total_hidden_states)

3503

In [ ]:
total_hidden_states[0]

array([-5.85052431e-01,  5.86862862e-02, -3.66477072e-01, -7.51274347e-01,
        4.02657211e-01, -1.25004426e-01,  1.68806449e-01,  8.05633605e-01,
       -6.08906865e-01, -4.64727640e-01,  2.89369285e-01, -2.61525393e-01,
       -5.37107706e-01,  6.77933216e-01,  1.69721290e-01,  4.75760251e-01,
       -1.81987032e-01,  2.39260316e-01,  2.44472548e-01,  2.47112140e-01,
       -7.73354620e-02, -5.15779495e-01,  5.24080396e-02,  4.84227180e-01,
        9.44615006e-01, -2.89410561e-01,  4.89270508e-01,  4.56486255e-01,
        3.12555403e-01, -1.34547547e-01, -1.89567074e-01,  2.95588598e-02,
       -1.24909937e+00, -2.99368620e-01, -5.72495818e-01,  1.26534263e-02,
       -3.04142386e-01, -5.13792515e-01, -5.70017219e-01,  5.50049722e-01,
       -2.95137018e-01, -4.27128315e-01,  3.83941323e-01,  5.98135553e-02,
        1.01590417e-01, -9.76150870e-01, -1.68274069e+00, -1.49207145e-01,
        4.15102005e-01,  2.47796521e-01,  1.22252107e-01,  1.28246337e-01,
       -1.00090647e+00, -

In [ ]:
import pandas as pd 
ths = pd.DataFrame(total_hidden_states)
ths

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.585052,0.058686,-0.366477,-0.751274,0.402657,-0.125004,0.168806,0.805634,-0.608907,-0.464728,...,-0.385121,0.080276,-0.044342,-0.321730,-0.767008,0.209783,0.383695,-0.067260,-0.788744,-0.158919
1,-0.413700,0.176161,0.365363,-0.564008,0.462006,-0.362096,0.073881,1.380798,-1.067544,-0.638728,...,-0.788935,0.484297,-0.150851,-0.275536,-0.503556,0.150069,0.463057,0.025615,-0.775409,-0.359056
2,-0.902619,-0.147502,0.118180,-0.644858,0.169539,-0.170470,0.162087,0.672882,-0.742499,-0.177525,...,-0.632766,0.093806,-0.102356,-0.498645,-0.727668,-0.134489,0.438116,0.245442,-0.855120,-0.267606
3,-0.393455,0.006274,0.372671,-0.493056,0.316550,-0.199721,0.640700,0.878999,-0.598908,-0.769553,...,-0.812874,0.362586,-0.084570,-0.539284,-0.672958,0.293768,0.597842,0.124103,-0.625668,-0.674369
4,-0.594593,0.058902,0.213317,-0.558663,0.600770,-0.394237,0.647671,0.738019,-0.499885,-0.595157,...,-0.789006,0.221974,-0.351420,-0.653864,-0.994403,0.250214,0.510145,0.111739,-0.883338,-0.270106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3498,0.016637,-0.040398,0.629643,-0.366519,0.317398,-0.004646,0.490902,1.039952,-0.633986,-1.024630,...,-0.628280,0.047093,-0.540411,-0.481455,-0.410564,0.180348,0.598770,0.088827,-0.208047,-0.624169
3499,-0.681916,-0.057709,0.436417,-0.626816,0.776405,-0.400645,0.231346,1.048974,-0.445948,-0.291516,...,-0.686142,-0.065003,-0.560170,-0.463045,-1.109406,0.186760,0.518436,0.232097,-0.912533,-0.527367
3500,-0.152477,0.869206,-0.323683,0.006687,0.317552,-0.362228,-0.156224,1.335583,-0.625530,0.061083,...,-0.164077,0.189320,-0.323539,-0.435027,-0.234720,-0.761106,-0.175736,0.028455,0.255223,-0.039689
3501,-0.657722,-0.045244,0.309572,-0.621982,0.715794,-0.447041,0.280048,0.928174,-0.652587,-0.373326,...,-0.570184,0.395047,-0.344805,-0.668759,-0.954096,0.376127,0.665060,-0.091239,-0.789942,-0.355522


In [ ]:
np.savetxt("/content/drive/MyDrive/PAPI_NASONOV/lab2/plot_hidden_states_distilbert_imdb.csv", ths, delimiter=",")

In [ ]:
ths_cpy = pd.read_csv("/content/drive/MyDrive/PAPI_NASONOV/lab2/plot_hidden_states.csv", header=None)
ths_cpy

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.086506,0.082164,-0.001604,-0.079234,0.182057,-0.083642,-0.033976,0.031674,0.054839,-0.055133,...,-0.012878,0.002627,-0.116417,-0.029677,0.031510,0.144877,0.003089,-0.053382,-0.019140,-0.064672
1,-0.125976,0.089755,0.009200,-0.102970,0.050067,-0.044505,0.001102,0.043528,0.072007,-0.097298,...,0.021123,-0.000165,-0.123041,-0.076145,0.059995,0.132067,-0.042424,-0.085623,-0.059184,-0.070154
2,-0.109840,0.086866,0.023348,-0.080036,0.148931,-0.001402,-0.057343,0.023224,0.091821,-0.076430,...,0.009983,0.039713,-0.135398,-0.090617,0.015318,0.103015,0.021311,-0.046125,0.009365,-0.019686
3,-0.110749,0.071028,-0.011592,-0.124983,0.146622,-0.030197,-0.025212,-0.033647,0.009671,-0.100500,...,0.027886,-0.020946,-0.097259,-0.129408,-0.077462,0.131632,0.002924,-0.089308,0.028985,-0.034490
4,-0.061950,0.096199,0.022849,-0.101171,0.086516,-0.046498,-0.003190,-0.003950,0.030579,-0.056033,...,-0.026434,0.037748,-0.051458,-0.116294,0.091911,0.123843,0.041377,-0.064117,-0.011405,-0.025460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3498,-0.057660,0.092983,0.016830,-0.105791,0.103354,-0.056901,-0.009661,-0.014314,0.028871,-0.067867,...,-0.022178,-0.003943,-0.040049,-0.074130,0.028132,0.130283,0.043754,-0.084529,-0.002873,-0.088836
3499,-0.090969,0.076540,-0.039789,-0.098358,0.151224,0.026478,-0.008637,0.025678,0.056332,-0.143283,...,0.007593,0.020992,-0.101840,-0.114824,-0.006660,0.143049,0.035252,-0.089942,0.003258,-0.091678
3500,-0.097047,0.061021,-0.010040,-0.134120,0.118408,-0.075939,-0.020949,0.071321,0.023173,-0.113103,...,-0.000272,0.010077,-0.060332,-0.067325,0.003528,0.160560,0.015210,-0.028311,-0.086733,-0.058869
3501,-0.109597,0.085818,-0.002590,-0.114537,0.160905,-0.084046,-0.040169,-0.008836,0.027442,-0.148681,...,0.027830,0.014315,-0.121199,-0.079290,-0.018222,0.132207,-0.052052,-0.120335,-0.001976,-0.021488
